# ESTACIONES HIDROMÉTRICAS OPTIMAS

In [1]:
from __future__ import print_function
import arcpy
from itertools import combinations
import numpy as np
import pandas as pd
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: column;
}
"""

HTML('<style>{}</style>'.format(CSS))

Se define el archivo shapefile que contiene a als estaciones hidrometricas

In [2]:
ehidrometricas = r'D:\SENAMHI\ehidrometrica\ehidrometrica\static\ehidrometrica.gdb\EH_GPT_EstacionHidrometrica_v7'

Se definen los nombres de los campos del shapefile de Estaciones Hidrométricas que seran utilizados durante el proceso

In [3]:
_L_OPT = "L_OPT"         # Longitud optima definida por la metodologia de Karasiev
_IDRC = "IDRC"           # Identificador de las region-cuenca
_EHIDROID = "EHIDROID"   # Identificador de cada estacion hidrometrica

Se obtienen e data frame donde se optiene la longitud optima

In [4]:
loptima = pd.read_csv('estaciones_optimas.csv')

_LOPT = 'lopt'
_IDCR = 'idcr'

def get_longitud_optima(idrc):
    response = loptima[loptima[_IDCR]==idrc][_LOPT]
    return response.iloc[0]

In [5]:
loptima.head()

,a,cv,eopt,flagLo,gyo,idcr,lcor,lgrad,lo,lopt,lrio,o,std,yo
0,0.000396,0.224210,11.0,Distancia,119.093364,000499410A,2008.449978,58.281931,2524.12310,58.281931,626.988905,0.2,2751.067569,12270.054897
1,0.113534,0.470814,7.0,Distancia,14.338734,001375608P,1.589399,21.165013,8.80790,11.377206,79.598245,0.2,252.582954,536.481041
2,0.070605,0.329343,6.0,Gradiente,2045.556964,000498107A,5.223128,32.071228,14.16340,18.647178,114.130341,0.2,38194.479342,115971.741887
3,3.392586,0.420917,3.0,0,0.000000,013753406P,0.000000,13.103706,0.29476,13.103706,39.311119,0.2,65.053272,154.551420
4,0.020573,1.101046,36.0,Distancia,40.597727,004979405A,1.603775,22.610954,48.60650,12.107364,435.040967,0.2,1786.697889,1622.727581


Se obtienen todos los IDRC

In [6]:
all_idrc = list(set([i[0] for i in arcpy.da.SearchCursor(ehidrometricas, [_IDRC])]))
all_idrc = [u'000013608P', u'000499205A', u'000499812A', u'000499912A', u'004664613A', u'004664910A', u'004929913A', u'004979405A', u'004989915A', u'004991211A']
print(all_idrc)

[u'000013608P', u'000499205A', u'000499812A', u'000499912A', u'004664613A', u'004664910A', u'004929913A', u'004979405A', u'004989915A', u'004991211A']


Definiendo variables globales para el proceso

In [7]:
aceptados = list()
result = list()
longitud_optima = float()
primer_filtro = list()
factor = 1

Definicion de funciones para el proceso

In [8]:
def evaluador(*args):
    """
    Determina aquellas estaciones hidrometricas de un IDRC que se encuentran 
    separadas a una mayor o igual distancia_optima
    """
    global longitud_optima
    global factor
    a, b = args[0]
    d = a[-1].distanceTo(b[-1])/1000
    if d >= longitud_optima*factor:
        return [a[0], b[0], d]
    else:
        return False


def buscar_optimos(*args):
    """
    Determina todas las soluciones posibles de agrupacion entre estaciones hidrometricas
    que se encuentran separadas a una mayor o igual distancia_optima
    """
    global aceptados
    global longitud_optima
    global primer_filtro
    ini, fin, dist = args
    for n in filter(lambda x: sorted(x) != sorted(list(args)), primer_filtro):
        if fin in n:
            ini_tmp = n[1] if n.index(fin) else n[0]
            fin_tmp = n[0] if n.index(fin) else n[1]
            if fin_tmp in list(set([q for x in aceptados for q in x[:2]])):
                continue
            dist_tmp = n[-1]
            controlador = 1
            for p in list(set([q for x in aceptados for q in x[:2]])):
                val = [l for l in primer_filtro if p in l and fin_tmp in l]
                if not val:
                    controlador = 0
                    break
            if controlador:
                aceptados.append(n)
                buscar_optimos(ini_tmp, fin_tmp, dist_tmp)   # recursividad

                
def get_estaciones_optimas(idrc):
    """
    Realiza la evaluacion de un IDRC para determinar la mejor solucion
    """
    try:
        global result
        global longitud_optima
        global soluciones
        global primer_filtro
        global aceptados
        print('Evaluando: {}'.format(idrc))

        query = "{} = '{}'".format(_IDRC, idrc)
        src = arcpy.SpatialReference(32718)
        geom = [i for i in arcpy.da.SearchCursor(ehidrometricas, [_EHIDROID, "SHAPE@", _L_OPT], query, src)]
        longitud_optima = get_longitud_optima(idrc)

        geom = map(lambda x: (x[0], x[1]), geom)

        combinaciones = list(combinations(geom, 2))

        primer_filtro = filter(lambda i: i, map(evaluador, combinaciones))
        
        for i in primer_filtro:
            ini, fin, dist = i
            aceptados.append(i)
            buscar_optimos(ini, fin, dist)
            soluciones.append(aceptados[:])
            aceptados = list()
        
        # Se obtiene la desviacion estandar de cada solucion en una lista, junto al index
        std_distancia = map(lambda i: [soluciones.index(i), np.std(map(lambda x:x[-1], i))], soluciones)
        
        # Se ordena de menor a mayor en base a la desviacion estandar
        std_distancia.sort(key=lambda i: i[-1], reverse=False)
        
        # Se obtiene el primer valor como solucion del proceso, ya que esto significa que la distancia
        # entre estaciones es la mas homogene
        solucion = soluciones[std_distancia[0][0]]
        
        # Se extraen los EHIDROID que pertenecen al index seleccionado
        optimos_list = list(set([q for x in solucion for q in x[:2]]))

        for i in optimos_list:
            result.append({_EHIDROID: i, 'OPT': 1})
            
        return {_IDRC: idrc, 'msg': 'success'}
    except Exception as e:
        return {_IDRC: idrc, 'msg': e.message.__str__()}
    finally:
        primer_filtro = list()
        soluciones = list()

Ejecutando el proceso masivo

In [9]:
logs = map(get_estaciones_optimas, all_idrc)

Evaluando: 000013608P


c:\python27\arcgis10.5\lib\site-packages\ipykernel_launcher.py:91: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


Evaluando: 000499205A
Evaluando: 000499812A
Evaluando: 000499912A
Evaluando: 004664613A
Evaluando: 004664910A
Evaluando: 004929913A
Evaluando: 004979405A
Evaluando: 004989915A
Evaluando: 004991211A


Exportando archivos excel de estaciones hidrometricas optimas y logs de procesamiento

In [10]:
df_optimos = pd.DataFrame(result)
display(df_optimos)
df_optimos.to_excel('estaciones_hidrometricas_optimas.xls')

df_logs = pd.DataFrame(logs)
display(df_logs)
df_logs.to_excel('logs.xls')
result= list() 

,EHIDROID,OPT
0,04591,1
1,04480,1
2,04236,1
3,04599,1
4,04717,1
5,04557,1
6,04047,1
7,04382,1
8,04121,1
9,04290,1


,IDRC,msg
0,000013608P,global name 'soluciones' is not defined
1,000499205A,success
2,000499812A,success
3,000499912A,success
4,004664613A,success
5,004664910A,success
6,004929913A,success
7,004979405A,success
8,004989915A,success
9,004991211A,success
